## ***01*** - Importing Dependencies

In [1]:
import os
import cv2
import numpy as np
#import matplotlib.pylot as plt
import pandas as pd
import tensorflow as pd

In [2]:
from keras.models import Sequential, Model
from keras.applications.xception import Xception

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten


Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.layers.normalization import BatchNormalization

In [4]:
from keras.models import Sequential, Model

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D,GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import backend as K

from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical


In [5]:
!pip install livelossplot
from livelossplot import PlotLossesKeras

## ***02*** - Output Files / Parameters / Hyperparameters

In [6]:
### OUTPUT FILES
train_data = '../input/chest_xray/chest_xray/train/'
val_data = '../input/chest_xray/chest_xray/val'
test_data = '../input/chest_xray/chest_xray/test'

normal_data_dir = '../input/chest_xray/chest_xray/train/NORMAL/'
pneumonia_data_dir = '../input/chest_xray/chest_xray/train/PNEUMONIA/'

## ***03*** - Helper Function

In [7]:
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 5.0
    lrate = initial_lrate * math*pow(drop, math.floor(1+epoch)/epoch_drop)
    return lrate

def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

## ***04*** - Data Augmentation / Data Generator

In [8]:
augs = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_augs = ImageDataGenerator(rescale=1./255)
train_gen = augs.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary')

test_gen = val_augs.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary')

val_gen = val_augs.flow_from_directory(
    val_data,
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


## ***05*** - Parameters / Hyperparameters

In [9]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNEL = 3

IMG_SIZE = IMG_HEIGHT, IMG_WIDTH
IMG_SIZES = IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL

NUM_EPOCH = 75
STEPS_PER_EPOCH = len(train_gen) * 2   #steps per epoch 216
VALIDATION_STEPS = len(val_gen) * 20   #Validations steps 16

## ***05*** - Callbacks

In [10]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,  TensorBoard, CSVLogger, LearningRateScheduler, ReduceLROnPlateau

model_checkpoint = ModelCheckpoint('./base.model',
                                    monitor='val_loss',
                                    verbose=1,
                                    save_best_only=True,
                                    mode='max',
                                    save_weights_only=False,
                                    period=1)

earlystop = EarlyStopping(monitor='val_loss',
                            min_delta=0.001,
                            patience=30,
                            verbose=1,
                            mode='auto')

tensorboard = TensorBoard(log_dir = './logs',
                            histogram_freq=0,
                            batch_size=16,
                            write_graph=True,
                            write_grads=True,
                            write_images=False)

csv_logger = CSVLogger(filename= "training_csv.log",
                        separator = ",",
                        append = False)

lrsched = LearningRateScheduler(step_decay,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                factor=0.8,
                                patience=5,
                                verbose=1, 
                                mode='auto',
                                min_delta=0.0001, 
                                cooldown=1, 
                                min_lr=0.0001)

callbacks = [model_checkpoint, earlystop, tensorboard, csv_logger, reduce_lr]

## ***06*** - Model Architecture

## ***06.1*** - ResNet152

In [11]:
from keras.applications.densenet import DenseNet169
base_1 = DenseNet169(include_top=False, 
                   weights='imagenet', 
                   input_shape=IMG_SIZES)

Instructions for updating:
Colocations handled automatically by placer.
51879936/51877672 [==============================] - 0s 0us/step


In [12]:
for layer in base_1.layers[:55]:
    layer.trainable = False

'''for layer in base_1.layers:
    print(layer, layer.trainable)'''

'for layer in base_1.layers:\n    print(layer, layer.trainable)'

In [14]:
model_1 = Sequential()

model_1.add(base_1)
model_1.add(GlobalAveragePooling2D())
model_1.add(Dense(1024, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1024, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model_1.compile(loss='binary_crossentropy', 
                    optimizer='adam', metrics=['accuracy'])

history_1 = model_1.fit_generator(train_gen, epochs=NUM_EPOCH, 
                                    steps_per_epoch=STEPS_PER_EPOCH, validation_data=test_gen, 
                                    validation_steps=VALIDATION_STEPS, callbacks=callbacks, verbose=1)

NameError: name 'model_1' is not defined

## ***06.2*** - InceptionV3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
base_2 = InceptionV3(include_top=False, 
                     weights='imagenet', 
                     input_shape=input_shape)

In [ ]:
for layer in base_2.layers[:87]:
    layer.trainable = False

'''for layer in base_2.layers:
    print(layer, layer.trainable)'''

In [ ]:
model_2 = Sequential()

model_2.add(base_2)
model_2.add(GlobalAveragePooling2D())
model_2.add(Dense(1, activation='sigmoid'))

In [ ]:
model_2.compile(loss='binary_crossentropy', 
                    optimizer='adam', metrics=['accuracy'])

history_2 = model_2.fit_generator(train_gen, epochs=75, 
                                    steps_per_epoch=300, validation_data=test_gen, 
                                    validation_steps=300, callbacks=callbacks, verbose=1)

## ***06.3*** - NasNetLarge

In [ ]:
from keras.applications.nasnet import NASNetLarge
base_3 = NASNetLarge(include_top='False', 
                     weights='imagenet')

In [ ]:
for layer in base_2.layers[:87]:
    layer.trainable = False

'''for layer in base_2.layers:
    print(layer, layer.trainable)'''

In [ ]:
model_3 = Sequential()
model_3.add(Dense(1, activation='sigmoid'))

In [ ]:
model_3.compile(loss='binary_crossentropy', 
                    optimizer='adam', metrics=['accuracy'])

history_3 = model_3.fit_generator(train_gen, epochs=75, 
                                    steps_per_epoch=300, validation_data=test_gen, 
                                    validation_steps=300, callbacks=callbacks, verbose=1)

## ***06.4*** - InceptionResNetV2

In [ ]:
'''from keras.applications.inception_resnet_v2 import InceptionResNetV2
base_6 = InceptionResNetV2(include_top=False, 
                           weights='imagenet', 
                           input_shape=input_shape)'''

In [ ]:
'''model_1 = Sequential()

model_1.add(base_1)
model_1.add(GlobalAveragePooling2D())
model_1.add(Dense(1024, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1024, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1, activation='sigmoid'))'''

In [ ]:
'''model_6.compile(loss='binary_crossentropy', 
                    optimizer='adam', metrics=['accuracy'])

history_6 = model_6.fit_generator(train_gen, epochs=75, 
                                    steps_per_epoch=300, validation_data=test_gen, 
                                    validation_steps=300, callbacks=callbacks, verbose=1)'''

## ***07*** - Showing the Result / Plotting the Model / Evaluation

In [ ]:
#show_final_history(history)